<a href="https://colab.research.google.com/github/JDkuba/FashionMnist/blob/master/Notebooks/IRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Synchronizacja z dyskiem
from google.colab import drive
drive.mount("/content/drive")

%cd './drive/My Drive/Projekt/FashionMnist'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Projekt/FashionMnist


In [2]:
#Importy i sprawdzenie czy mamy CUDA
import torch
import torchvision

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
dev

'cuda:0'

In [3]:
# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "SGD",
#               "optimizer_params": {"lr": 0.02, "momentum": 0.9},
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.1"}
# 32 loss: 0.036, accuracy: 99.18%
# Accuracy of the network on test images: 91.56%

# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "SGD",
#               "optimizer_params": {"lr": 0.02, "momentum": 0.9, "weight_decay": 1e-4},
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.2"}
# 32 loss: 0.061, accuracy: 98.44%
# Accuracy of the network on test images: 91.39%

# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "SGD",
#               "optimizer_params": {"lr": 0.001, "momentum": 0.9, "weight_decay": 1e-4, 'nesterov': True},
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.3"}
# 32 loss: 0.120, accuracy: 96.55%
# Accuracy of the network on test images: 89.36%

# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "Adam",
#               "optimizer_params": {"lr": 0.02},
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.4"}
# 32 loss: 0.032, accuracy: 99.03%
# Accuracy of the network on test images: 90.87%

# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "Adam",
#               "optimizer_params": {"lr": 0.02, "weight_decay": 1e-4},
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.5"}
# 32 loss: 0.219, accuracy: 92.46%, time: 19:07:26
# Accuracy of the network on test images: 89.32%

# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "AdamW",
#               "optimizer_params": { },
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.6"}
# 32 loss: 0.070, accuracy: 97.71%, time: 19:35:06
# Accuracy of the network on test images: 90.86%

# parameters = {"batch_size": 128,
#               "n_blocks": 3,
#               "optimizer": "AdamW",
#               "optimizer_params": { "lr": 0.02 },
#               "patience": 1,
#               "factor": 0.1,
#               "n_epochs": 32,
#               "name": "IRCNN_v0.7"}
# 32 loss: 0.049, accuracy: 98.51%
# Accuracy of the network on test images: 90.61%

parameters = {"batch_size": 128,
              "n_blocks": 3,
              "optimizer": "SGD",
              "optimizer_params": {"lr": 0.02, "momentum": 0.9},
              "patience": 1,
              "factor": 0.1,
              "n_epochs": 32,
              "dropout": 0.4,
              "name": "IRCNN_v0.8"}
# 32 loss: 0.032, accuracy: 99.19%
# Accuracy of the network on test images: 91.89%

In [4]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

#katalog, w którym przechowujemy dane
root = "data"
batch_size = parameters['batch_size']

#transformacje wektorów wejściowych
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, ), (0.5, ))])

train_dataset = dsets.FashionMNIST(root=root,
                            train=True,
                            transform=transform,
                            download=True)

test_dataset = dsets.FashionMNIST(root=root,
                            train=False,
                            transform=transform)

trainloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                batch_size=batch_size,
                                                shuffle=True)

testloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=batch_size,
                                                shuffle=True)


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class RCL(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size):
    super(RCL, self).__init__()
    padding = 0 if kernel_size == 1 else 1
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
    self.bn2d1 = nn.BatchNorm2d(num_features=out_channels)
    self.convr = nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding)
    self.bn2dr = nn.BatchNorm2d(num_features=out_channels)

  def forward(self, x):
    conv1 = self.conv1(x)
    conv2 = self.convr(F.relu(self.bn2d1(conv1)))
    x1 = conv1.add(conv2)
    conv3 = self.convr(F.relu(self.bn2dr(x1)))
    x1 = conv1.add(conv3)
    conv4 = self.convr(F.relu(self.bn2dr(x1)))
    x1 = conv1.add(conv4)
    x1 = F.relu(self.bn2dr(x1))
    return x1

class Block(nn.Module):
  def __init__(self):
    super(Block, self).__init__()
    self.rcl1 = RCL(in_channels=64, out_channels=64, kernel_size=1)
    self.rcl2 = RCL(in_channels=64, out_channels=128, kernel_size=3)
    self.avgPool2d = nn.AvgPool2d(3, stride=1, padding=1)
    self.rcl3 = RCL(in_channels=64, out_channels=64, kernel_size=1)

  def forward(self, x):
    x1 = self.rcl1(x)
    x2 = self.rcl2(x)
    x3 = self.rcl3(self.avgPool2d(x))
    return torch.cat((x1, x2, x3), 1)


In [6]:
class IRCNN(nn.Module):
  def __init__(self, dropout=0.5):
    super(IRCNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=2)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
    self.bn2d2 = nn.BatchNorm2d(num_features=32)
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
    self.bn2d3 = nn.BatchNorm2d(num_features=64)

    self.block1 = Block()
    self.conv_b1 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.bn2d_b1 = nn.BatchNorm2d(num_features=64)
    self.maxPool2d_b1 = nn.MaxPool2d(3, stride=1, padding=1)
    self.dropout_b1 = nn.Dropout2d(p=dropout)

    self.block2 = Block()
    self.conv_b2 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.bn2d_b2 = nn.BatchNorm2d(num_features=64)
    self.maxPool2d_b2 = nn.MaxPool2d(3, stride=1, padding=1)
    self.dropout_b2 = nn.Dropout2d(p=dropout)

    self.block3 = Block()
    self.conv_b3 = nn.Conv2d(in_channels=256, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.bn2d_b3 = nn.BatchNorm2d(num_features=64)
    self.globalAvgPool = nn.AdaptiveAvgPool2d(1)
    self.dropout_b3 = nn.Dropout2d(p=dropout)

    self.dense1 = nn.Linear(64, 10)

  def forward(self, x):
    x1 = self.conv1(x)
    x1 = F.relu(self.bn2d2(self.conv2(x1)))
    x1 = F.relu(self.bn2d3(self.conv3(x1)))
    #
    x1 = self.block1(x1)
    x1 = self.conv_b1(x1)
    x1 = self.bn2d_b1(x1)
    x1 = self.maxPool2d_b1(x1)
    x1 = self.dropout_b1(x1)
    #
    x1 = self.block2(x1)
    x1 = self.conv_b2(x1)
    x1 = self.bn2d_b2(x1)
    x1 = self.maxPool2d_b2(x1)
    x1 = self.dropout_b2(x1)
    #
    x1 = self.block3(x1)
    x1 = self.conv_b3(x1)
    x1 = self.bn2d_b3(x1)
    x1 = self.globalAvgPool(x1)
    x1 = self.dropout_b3(x1)
    x1 = x1.view(x1.size(0), -1)
    return self.dense1(x1)

In [7]:
if not torch.cuda.is_available():
  raise Exception("Change Runtime Type!")

dropout = parameters['dropout'] if 'dropout' in parameters else 0.5
net = IRCNN(dropout=dropout)
net.to(dev)

save_path = "saved_models/" + parameters["name"]

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()

if parameters['optimizer'] == 'SGD':
  optimizer = optim.SGD(net.parameters(), **parameters['optimizer_params'])
if parameters['optimizer'] == 'Adam':
  optimizer = optim.Adam(net.parameters(), **parameters['optimizer_params'])
if parameters['optimizer'] == 'AdamW':
  optimizer = optim.AdamW(net.parameters(), **parameters['optimizer_params'])

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=parameters['factor'],
                              patience=parameters['patience'], verbose=True)


def accuracy(model, dataloader):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in dataloader:
          images, labels = data
          if torch.cuda.is_available():
            images = images.to(dev)
            labels = labels.to(dev)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  return 100 * correct / total

In [8]:
best_accuracy = 0
n_epochs = parameters['n_epochs']

results = [] # list of tuples (epoch, loss, accuracy)

for epoch in range(n_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_loss_n = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_loss_n += 1

    epoch_accuracy = accuracy(net, trainloader)
    current_loss = running_loss / running_loss_n
    results.append((epoch + 1, current_loss, epoch_accuracy))
    print((f"{epoch + 1} loss: {current_loss:.3f}, "
          f"accuracy: {epoch_accuracy:.2F}%, "))
    
    scheduler.step(epoch_accuracy)
    if best_accuracy < epoch_accuracy:
      best_accuracy = epoch_accuracy
      torch.save((net.state_dict(), results), save_path)

print('Finished Training')

1 loss: 0.692, accuracy: 81.47%, 
2 loss: 0.457, accuracy: 86.11%, 
3 loss: 0.385, accuracy: 87.83%, 
4 loss: 0.340, accuracy: 88.65%, 
5 loss: 0.311, accuracy: 90.33%, 
6 loss: 0.292, accuracy: 90.28%, 
7 loss: 0.268, accuracy: 91.38%, 
8 loss: 0.251, accuracy: 91.82%, 
9 loss: 0.240, accuracy: 92.29%, 
10 loss: 0.228, accuracy: 92.48%, 
11 loss: 0.214, accuracy: 92.67%, 
12 loss: 0.205, accuracy: 93.60%, 
13 loss: 0.193, accuracy: 93.91%, 
14 loss: 0.184, accuracy: 94.33%, 
15 loss: 0.177, accuracy: 94.77%, 
16 loss: 0.167, accuracy: 94.88%, 
17 loss: 0.155, accuracy: 95.45%, 
18 loss: 0.148, accuracy: 95.73%, 
19 loss: 0.136, accuracy: 95.78%, 
20 loss: 0.130, accuracy: 96.12%, 
21 loss: 0.120, accuracy: 96.38%, 
22 loss: 0.118, accuracy: 96.38%, 
23 loss: 0.111, accuracy: 96.71%, 
24 loss: 0.100, accuracy: 96.88%, 
25 loss: 0.098, accuracy: 97.47%, 
26 loss: 0.090, accuracy: 96.87%, 
27 loss: 0.085, accuracy: 96.83%, 
Epoch    27: reducing learning rate of group 0 to 2.0000e-03.
28

In [10]:
params, results = torch.load(save_path)
net.load_state_dict(params)
print(f'Accuracy of the network on test images: {accuracy(net, testloader):.2F}%')

Accuracy of the network on test images: 91.89%
